In [1]:
import datetime

from cogment_lab.actors import RandomActor, ConstantActor
from cogment_lab.envs.gymnasium import GymEnvironment
from cogment_lab.process_manager import Cogment
from cogment_lab.utils.runners import process_cleanup
from cogment_lab.utils.trial_utils import format_data_multiagent


In [2]:
# Cleans up potentially hanging background processes from previous runs
process_cleanup()

Processes terminated successfully.


In [3]:
logpath = f"logs/logs-{datetime.datetime.now().isoformat()}"

cog = Cogment(log_dir=logpath)

print(logpath)

logs/logs-2023-12-13T18:33:52.073150


/Users/ariel/PycharmProjects/cogment_lab/venv/lib/python3.10/site-packages/cogment/context.py:213: UserWarning: No logging handler defined (e.g. logging.basicConfig)
  warnings.warn("No logging handler defined (e.g. logging.basicConfig)")


In [4]:
# Launch an environment in a subprocess

cenv = GymEnvironment(
    env_id="CartPole-v1",
    render=True
)

await cog.run_env(env=cenv, 
                  env_name="cartpole",
                  port=9001, 
                  log_file="env.log")


True

In [5]:
# Launch two dummy actors in subprocesses

random_actor = RandomActor(cenv.env.action_space)
constant_actor = ConstantActor(0)

await cog.run_actor(actor=random_actor, 
                    actor_name="random", 
                    port=9021, 
                    log_file="actor-random.log")

await cog.run_actor(actor=constant_actor,
                    actor_name="constant",
                    port=9022,
                    log_file="actor-constant.log")


True

In [6]:
# Check what's running

cog.processes

{'cartpole': <SpawnProcess name='SpawnProcess-1' pid=16499 parent=16475 started>,
 'random': <SpawnProcess name='SpawnProcess-2' pid=16502 parent=16475 started>,
 'constant': <SpawnProcess name='SpawnProcess-3' pid=16505 parent=16475 started>}

In [7]:
MOUNTAIN_CAR_ACTIONS = ["no-op", "ArrowLeft", "ArrowRight"]
LUNAR_LANDER_ACTIONS = ["no-op", "ArrowRight", "ArrowUp", "ArrowLeft"]
PONG_ACTIONS = ["no-op", "ArrowUp", "ArrowDown"]
CARTPOLE_ACTIONS = ["no-op", "ArrowRight"]

# Change this if you use a different environment. Only discrete actions are supported for now.
# no-op is the default action when no key is pressed

actions = CARTPOLE_ACTIONS

await cog.run_web_ui(actions=actions, log_file="human.log", fps=10)

True

In [8]:
trial_id = await cog.start_trial(
    env_name="cartpole",
    session_config={"render": True},
    actor_impls={
        "gym": "random",
    },
)

data = await cog.get_trial_data(trial_id)


In [9]:
data["gym"].observations.shape

(19, 4)

In [11]:
await cog.cleanup()